# Falling Fruit

In [ ]:
location = pd.read_csv("/Users/linh/Downloads/develop_for_good/locations.csv", low_memory = False)

In [ ]:
fruits = pd.read_csv("/Users/linh/Downloads/develop_for_good/observation_phenology.csv")

In [ ]:
#full join 
ff = pd.merge(location,fruits, on = ['lat', 'lng'], how = 'outer')

In [ ]:
ff.shape

In [ ]:
ff.head()

In [ ]:
#ff.to_csv('/Users/linh/Downloads/develop_for_good/falling_fruit.csv')

# Climate and Season

In [ ]:
#pip install meteostat

In [3]:
# Import Meteostat library and dependencies
import pandas as pd 
import numpy as np 
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Monthly

In [4]:
ff = pd.read_csv("/Users/linh/Downloads/develop_for_good/falling_fruit.csv", low_memory = False)

In [5]:
ff = ff[['scientific_name','id', 'lat', 'lng','location_id', 'observation_id', 'date', 'taxonomic_rank', 'en_name']]

In [6]:
ff = ff[ff['scientific_name'].notna()]

In [7]:
ff['lat'] = ff['lat'].apply(float)
ff['lng'] = ff['lng'].apply(float)
ff['scientific_name'] = ff['scientific_name'].apply(str)

In [8]:
ff.head()

,scientific_name,id,lat,lng,location_id,observation_id,date,taxonomic_rank,en_name
13,Citrus x limon,107.0,37.409772,-122.136046,107.0,2823.0,2013-12-14,8.0,Lemon
39,Ficus carica,419.0,37.862816,-122.262840,419.0,51054.0,2021-10-03,8.0,Common fig
115,Rubus,495.0,37.871975,-122.262749,495.0,40603.0,2019-07-25,7.0,Blackberry
177,Citrus x limon,557.0,37.856464,-122.267471,557.0,47572.0,2021-01-11,8.0,Lemon
202,Citrus,582.0,37.864990,-122.266190,582.0,47720.0,2021-02-21,7.0,Lime


In [9]:
ff.shape

(25386, 9)

In [17]:
class ClimateData:
    "Growing Degree Day setup for each unique fruit"
    
    def __init__(self, en_name, scientific_name, lat, lng, ideal_temp):
        "Key aspects of fruit given"
        self._en_name = en_name 
        self._lat = lat
        self._lng = lng
        self._scientific_name = scientific_name
        self._location = Point(lat, lng)
        self._ideal_temp = ideal_temp
        #making self._climate 
        location = self._location 
        climate = Monthly(location, datetime(2022, 1, 1), datetime(2022, 12, 31))
        climate = climate.fetch()
        climate = climate.reset_index()
        self._climate = climate 

    def get_lat(self): 
        return self._lat 
    def get_long(self): 
        return self._lng
    def get_scientific_name(self): 
        return self._scientific_name 
    def get_ideal_temp(self): 
        return self._ideal_temp 
    def get_en_name(self): 
        return self._en_name
    def get_climate(self): 
        return self._climate
    
    def get_gdd(self): 
        d = self._climate

        p1 = ['2022-12-01', '2022-01-01', '2022-02-01']
        p2 = ['2022-03-01', '2022-04-01', '2022-05-01']
        p3 = ['2022-06-01','2022-07-01', '2022-08-01']
        p4 = ['2022-09-01', '2022-10-01', '2022-11-01']
        #assign quarters 
        season = []
        for i in range(len(d['time'])): 
            if str(d['time'][i])[0:10] in p1: 
                season.append('p1_12_02')
            elif str(d['time'][i])[0:10] in p2: 
                season.append('p2_03_05')
            elif str(d['time'][i])[0:10] in p3: 
                season.append('p3_06_08')
            elif str(d['time'][i])[0:10] in p4: 
                season.append('p4_09_11')
            else: 
                season.append('NA')
        d['season'] = season 
    
        #groupby and get tavg, tmin, tmax for quarter 
        tavg = d.groupby('season')['tavg'].mean().reset_index().set_index('season').round(1)
        tmin = d.groupby('season')['tmin'].min().reset_index().set_index('season')
        tmax = d.groupby('season')['tmax'].max().reset_index().set_index('season')
        gdd = pd.concat([tavg, tmin, tmax]).reset_index()
        gdd['harvest'] = np.where(gdd['tavg'] > self._ideal_temp, 1,0)
        #sum doesn't actually do anything since it's na values 
        gdd = gdd.groupby(['season']).sum()
        self._gdd = gdd.reset_index()
        return self._gdd
    

In [18]:
lemon = ClimateData(ff['en_name'].values[0], 
                    ff['scientific_name'].values[0],
                    ff['lat'].values[0], 
                    ff['lng'].values[0],
                    18)

In [19]:
lemon.get_gdd()

,season,tavg,tmin,tmax,harvest
0,p1_12_02,10.8,5.2,17.7,0
1,p2_03_05,14.9,8.6,22.3,0
2,p3_06_08,18.7,13.9,25.2,1
3,p4_09_11,15.9,4.9,26.8,0
